In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from autils import *

def load_datasets(ratio = 0.9):
    # load numpy arrays
    X, y = load_data()

    m = X.shape[0]

    # split into training and test data set
    training_size = int(ratio * m)

    indices = np.random.permutation(m)    
    training_idx, test_idx = indices[:training_size], indices[training_size:]
    training_idx.sort()
    test_idx.sort()

    training_dataset = TensorDataset(torch.Tensor(X[training_idx, :]),torch.LongTensor(y[training_idx, 0]))
    test_dataset = TensorDataset(torch.Tensor(X[test_idx, :]),torch.LongTensor(y[test_idx, 0]))
    
    return training_dataset, test_dataset, X, y

In [ ]:
training_data, test_data, X_orig, y_orig = load_datasets(ratio = 0.9)

# Create data loaders
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
X, y = next(iter(test_dataloader))
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20 * 20, 25),
            nn.ReLU(),
            nn.Linear(25, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logps = self.linear_relu_stack(x)
        return logps

model = NeuralNetwork().to(device)
print(f"model: {model}")

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #if batch % 100 == 0:
        #    loss, current = loss.item(), (batch + 1) * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.NLLLoss()

# Optimizers require the parameters to optimize and a learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
print("Done!")

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')

np.set_printoptions(precision=2)

In [ ]:
correct_count, all_count = 0, 0
for images,labels in test_dataloader:
  for i in range(len(labels)):
    img = images[i].view(1, 400)
    # Turn off gradients to speed up this part
    with torch.no_grad():
        pred = model(img)

    pred_label = pred.argmax(1)
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)
    plt.tight_layout()

In [ ]:
m = len(test_data)
random_index = np.random.randint(m)
images, labels = test_data[random_index]
print(f"images.shape: {images.shape}")
print(f"labels.shape: {labels.shape}")

img = images.view(1, 400)
# Turn off gradients to speed up this part
with torch.no_grad():
    pred = model(img)

# Output of the network are log-probabilities, need to take exponential for probabilities
ps = torch.exp(pred)
print("Predicted Digit =", pred.argmax(1).item())
print("True Digit =", labels.item())
view_classify(img.view(1, 20, 20), ps)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# You do not need to modify anything in this cell

m = len(test_data)

correct_count, all_count = 0, 0

fig, axes = plt.subplots(8,8, figsize=(5,5))
fig.tight_layout(pad=0.13,rect=[0, 0.03, 1, 0.91]) #[left, bottom, right, top]
widgvis(fig)
for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)

    images, labels = test_data[random_index]
    img = images.view(1, 400)
    with torch.no_grad():
        pred = model(img)
    
    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = images.view(20,20).T
    
    # Display the image
    ax.imshow(X_random_reshaped, cmap='gray')

    y = labels.item()
    yhat = pred.argmax(1).item()
        
    # Display the label above the image
    if y == yhat:
        ax.set_title(f"{y},{yhat}",fontsize=10)
        correct_count += 1
    else:
        ax.set_title(f"{y},{yhat}",fontsize=10, color="r")
    all_count += 1
    ax.set_axis_off()
fig.suptitle("y, yhat", fontsize=14)

print(f"Number Of Images Tested: {all_count}")
print(f"Number Of miss: {all_count - correct_count}")
print(f"Model Accuracy: {(correct_count/all_count) * 100.0}%.\n")
plt.show()
